In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import scipy as sp
import scipy.stats
from scipy.stats import binom
import pandas as pd
from IPython.display import display, HTML

The following functions are taken from the notebooks at https://github.com/pbstark/MX14/.

In [1]:
def ecdf(x):
    '''
       calculates the empirical cdf of data x
       returns the unique values of x in ascending order and the cumulative probabity at those values
       NOTE: This is not an efficient algorithm: it is O(n^2), where n is the length of x. 
       A better algorithm would rely on the Collections package or something similar and could work
       in O(n log n)
    '''
    theVals = sorted(np.unique(x))
    theProbs = np.array([sum(x <= v) for v in theVals])/float(len(x))
    if (theVals[0] > 0.0):
        theVals = np.append(0., theVals)
        theProbs = np.append(0., theProbs)
    return theVals, theProbs

def massartOneSide(n, alpha):
    ''' 
       tolerable misfit between the cdf and ecdf for a one-sided bound 
       at significance level alpha for an iid sample of size n
    '''
    return np.sqrt(-np.log(alpha)/(2.0*n))
    
    
def ksLowerMean(x, c):
    '''
       lower confidence bound for the mean of a nonnegative population
       x is an iid sample with replacement from the population
       c is the Massart constant for the desired coverage
    '''
    # find the ecdf
    vals, probs = ecdf(x)
    probs = np.fmin(probs+c, 1)   # This is G^-
    gProbs = np.diff(np.append([0.0], probs))  # pre-pend a 0 so that diff does the right thing; 
                                               # gProbs is the vector of masses
    return (vals*gProbs).sum()


def kaplanWaldLowerCI(x, cl = 0.95, gamma = 0.99, xtol=1.e-12, logf=True):
    '''
       Calculates the Kaplan-Wald lower 1-alpha confidence bound for the mean of a nonnegative random
       variable.
    '''
    alpha = 1.0-cl
    if any(x < 0):
        raise ValueError('Data x must be nonnegative.')
    elif all(x <= 0):
        lo = 0.0
    else:
        if logf:
            f = lambda t: (np.max(np.cumsum(np.log(gamma*x/t + 1 - gamma))) + np.log(alpha))
        else:
            f = lambda t: (np.max(np.cumprod(gamma*x/t + 1 - gamma)) - 1/alpha)
        xm = np.mean(x)
        if f(xtol)*f(xm) > 0.0:
            lo = 0.0
        else:
            lo = sp.optimize.brentq(f, xtol, np.mean(x), xtol=xtol) 
    return lo

Let's study the choice of gamma vs the lower bound. The first example is a pointmass at 1 and uniform[0,1] mixture distribution. Here, the mean is very close to 1.  Small values of gamma give more informative lower bounds (i.e. closer to the actual mean).

In [5]:
# Nonstandard mixture: a pointmass at 1 and a uniform[0,1]

ns = np.array([400])  # sample sizes
ps = np.array([0.9, 0.99, 0.999])    # mixture fraction, weight of pointmass
alpha = 0.05  # 1- (confidence level)
reps = 1.0e3   # just for demonstration
gamma = np.array([0.01, 0.1, 0.5, 0.9, 0.999])  # tuning constant in Kaplan-Wald
xtol = 1.0e-12

cols = ['mass at 1', 'pop mean', 'sample size']
cols.extend(['KW cov ' + str(g) for g in gamma])
cols.extend(['KW low ' + str(g) for g in gamma])

simTable = pd.DataFrame(columns=cols)

for p in ps:
    popMean = (1-p)*0.5 + p
    for n in ns:
        covK = np.zeros(len(gamma))
        lowK = np.zeros(len(gamma))

        for rep in range(int(reps)):
            sam = np.random.uniform(size=n)
            ptMass = np.random.uniform(size=n)
            sam[ptMass < p] = 1.0
            samMean = np.mean(sam)
            #
            for i in range(len(gamma)):
                kLow = kaplanWaldLowerCI(sam, cl = 1-alpha, gamma = gamma[i], xtol = xtol)
                covK[i] += (kLow <= popMean)
                lowK[i] += kLow
            #
            
        theRow = [p, popMean, n]
        theRow.extend([str(100*float(covK[i])/float(reps)) + '%' for i in range(len(gamma))])
        theRow.extend([str(round(lowK[i]/float(reps),4)) for i in range(len(gamma))])
        simTable.loc[len(simTable)] = theRow
#
ansStr =  '<h3>Simulated coverage probability and expected lengths of one-sided nonparametric confidence intervals ' +\
          'mixture of U[0,1] and pointmass at 1</h3>' +\
          '<strong>Nominal coverage probability ' + str(100*(1-alpha)) +\
          '%</strong>. <br /><strong>Estimated from ' + str(int(reps)) + ' replications.</strong>'

display(HTML(ansStr))
display(simTable)

,mass at 1,pop mean,sample size,KW cov 0.01,KW cov 0.1,KW cov 0.5,KW cov 0.9,KW cov 0.999,KW low 0.01,KW low 0.1,KW low 0.5,KW low 0.9,KW low 0.999
0,0.900,0.9500,400,100.0%,100.0%,98.0%,95.9%,96.3%,0.5422,0.8825,0.9287,0.9138,0.9028
1,0.990,0.9950,400,100.0%,100.0%,100.0%,100.0%,100.0%,0.5679,0.9252,0.9794,0.9846,0.9846
2,0.999,0.9995,400,100.0%,100.0%,100.0%,100.0%,100.0%,0.5706,0.9296,0.9847,0.9911,0.9918


Our second example is a mixture distribution with a point mass at 0 and a Poisson distribution with mean 10.  With so many 0 values, the mean of these distributions are small.  In contrast to the previous example, large values of gamma give uninformative lower bounds.

In [7]:
# Nonstandard mixture: a pointmass at 0 and a Poisson(10)

ns = np.array([400])  # sample sizes
ps = np.array([0.9, 0.99, 0.999])    # mixture fraction, weight of pointmass
alpha = 0.05  # 1- (confidence level)
reps = 1.0e3   # just for demonstration
gamma = np.array([0.01, 0.1, 0.5, 0.9, 0.999])  # tuning constant in Kaplan-Wald
xtol = 1.0e-12

cols = ['mass at 1', 'pop mean', 'sample size']
cols.extend(['KW cov ' + str(g) for g in gamma])
cols.extend(['KW low ' + str(g) for g in gamma])

simTable = pd.DataFrame(columns=cols)

for p in ps:
    popMean = (1-p)*10
    for n in ns:
        covK = np.zeros(len(gamma))
        lowK = np.zeros(len(gamma))

        for rep in range(int(reps)):
            sam = np.random.poisson(lam=10, size=n)
            ptMass = np.random.uniform(size=n)
            sam[ptMass < p] = 0.0
            samMean = np.mean(sam)
            #
            for i in range(len(gamma)):
                kLow = kaplanWaldLowerCI(sam, cl = 1-alpha, gamma = gamma[i], xtol = xtol)
                covK[i] += (kLow <= popMean)
                lowK[i] += kLow
            #
            
        theRow = [p, popMean, n]
        theRow.extend([str(100*float(covK[i])/float(reps)) + '%' for i in range(len(gamma))])
        theRow.extend([str(round(lowK[i]/float(reps),4)) for i in range(len(gamma))])
        simTable.loc[len(simTable)] = theRow
#
ansStr =  '<h3>Simulated coverage probability and expected lengths of one-sided nonparametric confidence intervals ' +\
          'mixture of Poisson(10) and pointmass at 0</h3>' +\
          '<strong>Nominal coverage probability ' + str(100*(1-alpha)) +\
          '%</strong>. <br /><strong>Estimated from ' + str(int(reps)) + ' replications.</strong>'

display(HTML(ansStr))
display(simTable)

,mass at 1,pop mean,sample size,KW cov 0.01,KW cov 0.1,KW cov 0.5,KW cov 0.9,KW cov 0.999,KW low 0.01,KW low 0.1,KW low 0.5,KW low 0.9,KW low 0.999
0,0.900,1.00,400,100.0%,97.3%,99.5%,100.0%,100.0%,0.5314,0.6195,0.1523,0.063,0.0538
1,0.990,0.10,400,98.7%,98.6%,99.7%,100.0%,100.0%,0.0294,0.0102,0.0017,0.0006,0.0
2,0.999,0.01,400,98.5%,99.3%,100.0%,100.0%,100.0%,0.0008,0.0003,0.0,0.0,0.0


Do things in an illegitimate way: optimize the Kaplan-Wald lower bound over choices of gamma. What happens